In [ ]:
#default_exp functional

In [ ]:
#exports
from kesscore.imports import *
from functools import wraps

In [ ]:
#exports
def zip_cycle_longest(*args):
    "Like `itertools.zip_longest` but `cycle`s through elements of all but longest argument"
    args = L(list(args)).map(list)
    n    = args.map(len).reduce(max)
    for i in range(n): yield tuple(a[i%len(a)] for a in args)

In [ ]:
test_eq(list(zip_cycle_longest([1,2],[3,4,5])), [(1,3),(2,4),(1,5)])

In [ ]:
#exports
def _wrapify(f, t):
    @wraps(f)
    def _inner(*args,**kwargs): return t(f(*args,**kwargs))
    return _inner
_listify = partial(_wrapify, t=list)
_Listify = partial(_wrapify, t=L)

In [ ]:
#exports
lmap = _listify(map)
lzip = _listify(zip)
lfilter = _listify(filter)
lrange = _listify(range)
Lmap = _Listify(map)
Lzip = _Listify(zip)
Lfilter = _Listify(filter)
Lrange = _Listify(range)

In [ ]:
test_eq_type(lmap(noop, [1,2,3]),          [1,2,3])
test_eq_type(lzip([1,2],[3,4]),            [(1,3),(2,4)])
test_eq_type(lfilter(ge(2), [1,2,3,4]),    [2,3,4])
test_eq_type(lrange(3),                    [0,1,2])
test_eq_type(Lmap(noop, [1,2,3]),          L([1,2,3]))
test_eq_type(Lzip([1,2],[3,4]),            L([(1,3),(2,4)]))
test_eq_type(Lfilter(ge(2), [1,2,3,4]),    L([2,3,4]))
test_eq_type(Lrange(3),                    L([0,1,2]))

In [ ]:
#exports
def _all(self:L)->bool: return all(self)
L.all = _all

In [ ]:
assert not L([True, False, True] ).all()
assert not L([True, True,  False]).all()
assert     L([True, True,  True] ).all()

In [ ]:
#exports
@patch
def all_eq(self:L)->bool: return all([self[i] == self[i-1] for i in range(1, len(self))])

In [ ]:
assert L(1,1,1,1).all_eq()
assert not L(1,10).all_eq()
assert L(1, True).all_eq()

In [ ]:
#exports
_isinstance = isinstance
def isinstance(obj, class_or_tuple=None):
    'Like isinstance but support creating partial versions, e.g, isinstance(int)(1) is True.'
    if class_or_tuple is None:
        class_or_tuple = obj
        def _inner(obj): 
            return _isinstance(obj, class_or_tuple)
        _inner.__doc__ = f'Check if `obj` is of type\\s: {class_or_tuple}'
        return _inner
    return _isinstance(obj,class_or_tuple)

In [ ]:
assert isinstance(obj=1, class_or_tuple=int)
assert isinstance(int)(1)
test_fail(lambda: isinstance(1)(int), contains='isinstance() arg 2 must be a type or tuple of types')
assert isinstance(int)(1)
assert all(map(isinstance((list, int)), [1, 2, 3, [1, 2, 3]]))
assert not isinstance((list, int))((1, 2, 3))

In [ ]:
#exports
def compose_star(*funcs):
    'like compose but pass *x to functions'
    def _inner(*x):
        for f in funcs: x = f(*x)
        return x
    return _inner

In [ ]:
def x(a,b): return a-1,b+1
test_eq(compose_star(x,x,x)(5,0), (2,3))

In [ ]:
#exports
@patch
def group_by(self:L, key=noop):
    res = dict()
    for v in self: res.setdefault(key(v), L()).append(v)
    return res

In [ ]:
test_eq_type(L([1,2,3,4]).group_by(lambda x:x%2), {0:[2,4], 1:[1,3]})

In [ ]:
#exports
def add_kwargs(**kwargs):
    "Decorator: add argument with default value to `**kwargs` in both signature and function"
    def _f(f):
        @wraps(f)
        def _inner(*args, **kw): return f(*args, **{**kwargs, **kw})
        sig = inspect.signature(_inner)
        sigd = dict(sig.parameters)
        for k,v in kwargs.items():
            if k in sigd.keys(): 
                assert sigd[k].kind.name not in ['POSITIONAL_ONLY', 'VAR_KEYWORD', 'VAR_POSITIONAL'], \
                       f'cannot assign an existing variable ({k!r}) of type {sigd[k].kind.name}'
                sigd[k] = sigd[k].replace(default=v, kind=inspect._ParameterKind.KEYWORD_ONLY)
            else: sigd[k] = inspect.Parameter(k, inspect._ParameterKind.KEYWORD_ONLY, default=v)
        params = [[p for p in sigd.values() if p.kind == t] for t in range(5)]
        _inner.__signature__ = sig.replace(parameters=concat(*params))
        return _inner
    return _f

In [ ]:
def a(x:float, /, y:NoneType, *args:int, z:int, t:str=3, **kwargs:str): '''Hey there :)'''; pass
doc(a)

a [source] a ( x : float , y : NoneType , * args : int , z : int , t : str = 3 , ** kwargs : str ) 
 
 Hey there :)

In [ ]:
test_fail(lambda:add_kwargs(x=3)(a), contains="cannot assign an existing variable ('x') of type POSITIONAL_ONLY")

In [ ]:
@add_kwargs(i=3, t=2)
def a(x:float, /, y:NoneType, *args:int, z:int, t:str=3, **kwargs:str): 
    '''Hey there :)'''
    return f'{x=},{y=},{args=},{z=},{t=},{kwargs=}'
test_eq(f'{inspect.signature(a)!r}', '<Signature (x: float, /, y: NoneType, *args: int, z: int, t: str = 2, i=3, **kwargs: str)>')
test_eq(a(1,2,3,4,z=5,v=6),"x=1,y=2,args=(3, 4),z=5,t=2,kwargs={'i': 3, 'v': 6}")

In [ ]:
#exports
def get_all_public_attr(o): 'Return all public field\\funcs'; return {k:getattr(o,k) for k in dir(o) if not k.startswith('_')}

In [ ]:
class A:
    def __init__(self):self.i,self._x,self.__x=1,2,3
    def f(self): print('hey')
    def _f(self):print('help')
test_eq(get_all_public_attr(A()).keys(), ['f','i'])

In [ ]:
#exports
def pcolkw(f, *, use_locals=True, use_kwargs=True, use_globals=False):
    '''Partial Collected kwargs: Call function with relevant arguments collected from frame above. (global->local->kwargs_outer->kwargs)'''
    params = inspect.signature(f).parameters.values()
    for p in params: 
        assert p.kind.name not in ['POSITIONAL_ONLY', 'VAR_POSITIONAL', 'VAR_KEYWORD'], f'Unsupported parameter in function: {p.name!r} is a {p.kind.name} parameter.'
    params = L(params).map(lambda p: p.name)
    
    fr = sys._getframe(1)
    
    def filter_rel(d): return {k:v for k,v in d.items() if k in params}
    globs  = filter_rel(fr.f_globals) if use_globals else {}
    locls  = filter_rel(fr.f_locals)  if use_locals else {}
    kwargs_outer = filter_rel(fr.f_locals['kwargs']) if use_kwargs and 'kwargs' in fr.f_locals.keys() else {}
    
    return partial(f, **{**globs, **locls, **kwargs_outer})

In [ ]:
def f(x ,/, y, *args, ido, guy=1, **kw):pass
def x(y, ido): pcolkw(f)()
test_fail(lambda:x(1,2), contains="Unsupported parameter in function: 'x' is a POSITIONAL_ONLY parameter")
def f(y, *args, ido, guy=1, **kw):pass
test_fail(lambda:x(1,2), contains="Unsupported parameter in function: 'args' is a VAR_POSITIONAL parameter")
def f(y, *, ido, guy=1, **kw):pass
test_fail(lambda:x(1,2), contains="Unsupported parameter in function: 'kw' is a VAR_KEYWORD parameter")

In [ ]:
def f(y, *, ido, guy=1):return f'{y=},{ido=},{guy=}'
def x(y, ido): return pcolkw(f)()
test_eq(x(1,2), 'y=1,ido=2,guy=1')
y=3
def x(ido): return pcolkw(f)()
test_fail(lambda:x(4), contains="f() missing 1 required positional argument: 'y'")
def x(ido): return pcolkw(f, use_globals=True)()
test_eq(x(4), 'y=3,ido=4,guy=1')
def x(ido, **kwargs): return pcolkw(f)()
test_eq(x(4, y=6), 'y=6,ido=4,guy=1')
def x(ido): return pcolkw(f)(5)
test_eq(x(4), 'y=5,ido=4,guy=1')
def x(ido): return pcolkw(f)(y=5)
test_eq(x(4), 'y=5,ido=4,guy=1')

In [ ]:
from nbdev.sync import notebook2script;notebook2script()

Converted 00_functional.ipynb.
Converted 01_tests.ipynb.
Converted 02_tensor.ipynb.
Converted 03_images.ipynb.
Converted 04_random.ipynb.
Converted 05_domainadaptation.ipynb.
Converted 06_mlp.ipynb.
Converted 07_download.ipynb.
Converted 08_script.ipynb.
Converted index.ipynb.
